# Проект 1. Предобработка данных

## Задание 1. Исследование данных

### Изучение информации о файле данных

Выводим файл и смотрим общую информацию о данных.

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Видно, что есть два столбца с вещественными значениями (days_employed и total_income), которые нужно заменить на целые числа.  
В этих же столбцах пропущенные значения,причем их число одинаковое, возможно, это одни и те же строки.  
Оценим структуру данных.

In [3]:
print(data.head(15))

    children  days_employed  dob_days            education  education_id  \
0          1   -8437.673028        42               высшее             0   
1          1   -4024.803754        36              среднее             1   
2          0   -5623.422610        33              Среднее             1   
3          3   -4124.747207        32              среднее             1   
4          0  340266.072047        53              среднее             1   
5          0    -926.185831        27               высшее             0   
6          0   -2879.202052        43               высшее             0   
7          0    -152.779569        50              СРЕДНЕЕ             1   
8          2   -6929.865299        35               ВЫСШЕЕ             0   
9          0   -2188.756445        41              среднее             1   
10         2   -4171.483647        36               высшее             0   
11         0    -792.701887        40              среднее             1   
12         0

Странности: 
- отрицательные значения для трудового стажа
- разные наименования для схожих целей (покупка недвижимости/покупка жилья/операции с жильем и т.д.)

### Заполнение пропущенных значений и замена типа данных

Но для начала разберемся с пропущенными значениями.  
Проверим гипотезу, что строки с отсутствующим уровнем дохода и без информации о трудовом стаже, - это одни и те же люди

In [4]:
print(data[data['days_employed'].isnull()].count())

children            2174
days_employed          0
dob_days            2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


Гипотеза подтвердилась! Но мы не знаем, в чем причина - не было ли дохода и опыта, или данные не были предоставлены.  
Попробуем по **income_type** понять, возможно ли такое, что могут быть в базе люди без дохода и опыта. Или данные просто не были предоставлены (не требовались при выдаче), и мы вправе заменить пропуски в этих количественных переменных на среднее арифметическое или медиану.

In [5]:
print(data[data['days_employed'].isnull()]['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Похоже, что данные не были предоставлены. Статус предполагает, что тот или иной уровень дохода должен быть.  
В таком случае определим, что лучше использовать для каждого из параметров - **среднее** или **медиану**.

In [6]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].median())
print(data['total_income'].mean())

20667.26379327158
2265604.028722744
145017.93753253992
167422.30220817294


Среднее и медиана близки, распределение позволяет использовать **среднее**.

In [7]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [10]:
total_income_grouped = data.groupby('income_type')['total_income'].mean().reset_index()
print(total_income_grouped)

       income_type   total_income
0      безработный  131339.751676
1        в декрете   53829.130729
2      госслужащий  170898.309923
3        компаньон  202417.461462
4        пенсионер  137127.465690
5  предприниматель  499163.144947
6        сотрудник  161380.260488
7          студент   98201.625314


In [16]:
print(data.head())

   children  days_employed  dob_days education  education_id  \
0         1   -8437.673028        42    высшее             0   
1         1   -4024.803754        36   среднее             1   
2         0   -5623.422610        33   Среднее             1   
3         3   -4124.747207        32   среднее             1   
4         0  340266.072047        53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  


In [22]:
def income_by_type(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if pd.isnull(total_income) == False:
        return total_income
    else:
        return total_income_grouped[total_income_grouped['income_type'] == income_type]['total_income'].values[0]
data['total_income_new'] = data.apply(income_by_type, axis=1)
print(data.head(20))

    children  days_employed  dob_days            education  education_id  \
0          1   -8437.673028        42               высшее             0   
1          1   -4024.803754        36              среднее             1   
2          0   -5623.422610        33              Среднее             1   
3          3   -4124.747207        32              среднее             1   
4          0  340266.072047        53              среднее             1   
5          0    -926.185831        27               высшее             0   
6          0   -2879.202052        43               высшее             0   
7          0    -152.779569        50              СРЕДНЕЕ             1   
8          2   -6929.865299        35               ВЫСШЕЕ             0   
9          0   -2188.756445        41              среднее             1   
10         2   -4171.483647        36               высшее             0   
11         0    -792.701887        40              среднее             1   
12         0

In [7]:
total_income_average = data['total_income'].mean()
data['total_income'] = data['total_income'].fillna(total_income_average)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_days            21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Заменим формат в столбце **total_income** на целое число. Некоторой неточностью округления в данном случае можно пренебречь.

In [8]:
data['total_income'] = data['total_income'].astype(int)
print(data.head(15))

    children  days_employed  dob_days            education  education_id  \
0          1   -8437.673028        42               высшее             0   
1          1   -4024.803754        36              среднее             1   
2          0   -5623.422610        33              Среднее             1   
3          3   -4124.747207        32              среднее             1   
4          0  340266.072047        53              среднее             1   
5          0    -926.185831        27               высшее             0   
6          0   -2879.202052        43               высшее             0   
7          0    -152.779569        50              СРЕДНЕЕ             1   
8          2   -6929.865299        35               ВЫСШЕЕ             0   
9          0   -2188.756445        41              среднее             1   
10         2   -4171.483647        36               высшее             0   
11         0    -792.701887        40              среднее             1   
12         0

In [9]:
print(data['days_employed'].min())
print(data['days_employed'].max())
print(data['days_employed'].median())
print(data['days_employed'].mean())

-18388.949900568383
401755.40047533
-1203.369528770489
63046.49766147338


Здесь загвоздка... Сначала необходимо разобраться со значениями в столбце **days_employed**, а затем уже заполнять пропуски. Проблемы две: отрицательные значения и слишком большие значения.  
Посчитаем количество отрицательных значений.

In [10]:
print(data[data['days_employed'] < 0]['days_employed'].count())

15906


Судя по количеству значений, ошибка системная. В идеале, необходимо обратиться к поставщикам информации и попытаться уточнить ситуацию. Но предположим, что ошибка только в знаке, и исправим ее.

In [11]:
def days_employed_group(value):
    if value < 0:
        return -1
    return 1
data['days_employed_group'] = data['days_employed'].apply(days_employed_group)
data['days_employed'] = data['days_employed'] * data['days_employed_group']
print(data.head(15))

    children  days_employed  dob_days            education  education_id  \
0          1    8437.673028        42               высшее             0   
1          1    4024.803754        36              среднее             1   
2          0    5623.422610        33              Среднее             1   
3          3    4124.747207        32              среднее             1   
4          0  340266.072047        53              среднее             1   
5          0     926.185831        27               высшее             0   
6          0    2879.202052        43               высшее             0   
7          0     152.779569        50              СРЕДНЕЕ             1   
8          2    6929.865299        35               ВЫСШЕЕ             0   
9          0    2188.756445        41              среднее             1   
10         2    4171.483647        36               высшее             0   
11         0     792.701887        40              среднее             1   
12         0

Сделано. Теперь попробуем перевести в дни в годы и посмотреть, в каком количестве случаев стаж превышает 50 лет.

In [12]:
data['years_employed'] = data['days_employed'] // 365
print(data[data['years_employed'] > 50]['years_employed'].count())

3445


Очень много... Что ж, пока оставим эту загадку, тем более, что у нас нет задачи проанализировать кредитоспособность в зависимости от стажа. Необходимо получить дополнительную информацию от поставщиков, а затем внести коррективы.  
В любом случае, заменим пропущенные значения и переведем в целочисленный формат.

In [13]:
print(data['days_employed'].min())
print(data['days_employed'].max())
print(data['days_employed'].median())
print(data['days_employed'].mean())

24.14163324048118
401755.40047533
2194.220566878695
66914.72890682236


Очевидно, что нужно использовать **медиану** - выпадающие чересчур высокие значения завышают **среднее**.

In [14]:
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['days_employed'] = data['days_employed'].astype(int) #переведем в целочисленные значения
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children               21525 non-null int64
days_employed          21525 non-null int64
dob_days               21525 non-null int64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null int64
purpose                21525 non-null object
days_employed_group    21525 non-null int64
years_employed         19351 non-null float64
dtypes: float64(1), int64(8), object(5)
memory usage: 2.3+ MB
None


### Удаление дубликатов

Начинаем поиск дубликатов.
Проверим столбцы **children**, **education**, **family_status**, **gender**, **income_type** на наличие дубликатов.  
Поскольку количество вариантов ответа в каждом столбце невелико, мы можем использовать метод *value_counts*.

In [15]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


-1 ребенок навевает грустные мысли... Скорее всего, это ошибка ввода, заменим на 1. Также, как и 20, скорее всего, 2.

In [16]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


С детьми разобрались. Посмотрим, что с образованием.

In [17]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Видимо, образование вводили вручную, отсюда дубликаты. Но похоже, что приведение к нижнему регистру должно решить проблему

In [18]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Заодно проверим, что корректно приписаны education_id

In [19]:
print(data.groupby('education').agg({'education_id': ['min', 'max']}))

                    education_id    
                             min max
education                           
высшее                         0   0
начальное                      3   3
неоконченное высшее            2   2
среднее                        1   1
ученая степень                 4   4


Ура! Взглянем на семейное положение.

In [20]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Все ОК

In [21]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Дубликатов нет, но есть одно странное значение... Кто знает, может быть, это третий пол))? В любом случае, можно оставить без изменений, поскольку это лишь один кейс, и анализ по полу не планируется

In [22]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


### Лемматизация

Приступим к лемматизации данных в столбце **purpose**.  
Для начала выведем все уникальные значения в столбце

In [23]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Видно, что есть несколько тем: приобретение или ремонт недвижимости, приобретение автомобиля, образование, свадьба. 
Но все написано по-разному, необходимо привести все к единому виду для последующей классификации.  
Создадим столбец с леммой для каждой строчки

In [24]:
from pymystem3 import Mystem
m = Mystem()
def lemma(text):
    return ' '.join(m.lemmatize(text))
data['purpose_lemma'] = data['purpose'].apply(lemma)
print(data.head(15))

    children  days_employed  dob_days            education  education_id  \
0          1           8437        42               высшее             0   
1          1           4024        36              среднее             1   
2          0           5623        33              среднее             1   
3          3           4124        32              среднее             1   
4          0         340266        53              среднее             1   
5          0            926        27               высшее             0   
6          0           2879        43               высшее             0   
7          0            152        50              среднее             1   
8          2           6929        35               высшее             0   
9          0           2188        41              среднее             1   
10         2           4171        36               высшее             0   
11         0            792        40              среднее             1   
12         0

In [26]:
print(data['purpose_lemma'].unique())

['покупка   жилье \n' 'приобретение   автомобиль \n'
 'дополнительный   образование \n' 'сыграть   свадьба \n'
 'операция   с   жилье \n' 'образование \n' 'на   проведение   свадьба \n'
 'покупка   жилье   для   семья \n' 'покупка   недвижимость \n'
 'покупка   коммерческий   недвижимость \n'
 'покупка   жилой   недвижимость \n'
 'строительство   собственный   недвижимость \n' 'недвижимость \n'
 'строительство   недвижимость \n'
 'на   покупка   подержать   автомобиль \n'
 'на   покупка   свой   автомобиль \n'
 'операция   с   коммерческий   недвижимость \n'
 'строительство   жилой   недвижимость \n' 'жилье \n'
 'операция   со   свой   недвижимость \n' 'автомобиль \n'
 'заниматься   образование \n' 'сделка   с   подержанный   автомобиль \n'
 'получение   образование \n' 'свадьба \n'
 'получение   дополнительный   образование \n' 'покупка   свой   жилье \n'
 'операция   с   недвижимость \n' 'получение   высокий   образование \n'
 'свой   автомобиль \n' 'сделка   с   автомобиль \n'
 'про

Получилось создать столбец с леммой для каждой строчки.  
Теперь создадим функцию и применим метод *apply()* для создания столбца с классификацией целей.

In [33]:
classification = { 'недвижимость': 'приобретение или ремонт недвижимости', 'жилье': 'приобретение или ремонт недвижимости', 'свадьба': 'свадьба', 'автомобиль': 'приобретение автомобиля', 'образование': 'образование' }
print(type(classification))

<class 'dict'>


In [39]:
def purpose_group(text):
    for key in classification:
        if key in text:
            return classification.get(key)
data['purpose_group'] = data['purpose_lemma'].apply(purpose_group)
print(data.head())

   children  days_employed  dob_days education  education_id  \
0         1           8437        42    высшее             0   
1         1           4024        36   среднее             1   
2         0           5623        33   среднее             1   
3         3           4124        32   среднее             1   
4         0         340266        53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose  days_employed_group  years_employed  \
0               покупка жилья                   -1            23

Все варианты удалось распределить по 4 категориям.

### Категоризация данных и создание словарей

Начнем с целей кредита, с которыми только что работали. Создадим словарь, который позволит позволит впоследствии создать более компактный датафрейм с данными, к которым мы будем обращаться.  
Для начала "оцифруем" категории

In [25]:
def purpose_category(purpose):
    if purpose == 'приобретение или ремонт недвижимости':
        return 1
    if purpose == 'свадьба':
        return 2
    if purpose == 'приобретение автомобиля':
        return 3
    if purpose == 'образование':
        return 4
data['purpose_id'] = data['purpose_group'].apply(purpose_category)
data['purpose_id'] = data['purpose_id'].astype(int)
print(data['purpose_id'].value_counts())

1    10840
3     4315
4     4022
2     2348
Name: purpose_id, dtype: int64


Создаем словарь и удаляем дубликаты

In [26]:
purpose_dict = data[['purpose_group','purpose_id']]
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
print(purpose_dict.sort_values('purpose_id'))

                          purpose_group  purpose_id
0  приобретение или ремонт недвижимости           1
3                               свадьба           2
1               приобретение автомобиля           3
2                           образование           4


Приступим к классификации данных и созданию словарей для трех других параметров, для которых планируется анализ кредитоспособности.   
Начнем с наличия детей.
Первым делом проверим, на какие категории имеет смысл разбивать 

In [27]:
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Наверное, можно было бы выделить отдельно группу семей с двумя и более детьми. Но это не очень целесообразно, поскольку эта группа была бы сравнительно небольшой. Выделим две категории: семьи с детьми и без детей

In [28]:
def children_group(number): 
    if number == 0:
        return 'Нет детей'
    return 'Есть дети'
data['children_group'] = data['children'].apply(children_group)
print(data['children_group'].value_counts())

Нет детей    14149
Есть дети     7376
Name: children_group, dtype: int64


Здесь всего два значения, нет особого смысла создавать словарь, с этой переменной будет работать достаточно просто.

Проанализируем ситуацию с семейным положением.

In [29]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Откровенно говоря, не очень понятно, как именно объединять группы по семейному положению. Дело в том, что каждая из групп может по-разному вести себя в отношении возврата кредита. Например, вдруг официальная регистрация брака позитивно влияет на возврат кредита, а клиенты, которые не торопятся ставить штамп в паспорте, менее ответственны и в других вопросах?  
Создадим словарь по имеющимся категориям, при необходимости сгруппируем на этапе анализа по сходству потребительского поведения.

In [30]:
family_status_dict = data[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict.sort_values('family_status_id'))

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


Проанализируем ситуацию с уровнем дохода. Цель - разбить на несколько крупных и примерно равных по размеру категорий.  
Для начала еще раз посмотрим на минимальное, максимальное, среднее и медианное значения.

In [31]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].median())
print(data['total_income'].mean())

20667
2265604
156400.0
167421.82174216027


Попробуем разбить на 4 равных группы, используя классификацию по процентильным группам

In [32]:
import numpy as np
cut_points = [np.percentile(data['total_income'], i) for i in [25, 50, 75]]
data['income_group_id'] = 1
for i in range(3):
    data['income_group_id'] = data['income_group_id'] + (data['total_income'] < cut_points[i])
print(data['income_group_id'].value_counts())

1    5382
4    5381
3    5381
2    5381
Name: income_group_id, dtype: int64


In [33]:
print(data.groupby('income_group_id').agg({'total_income':['min', 'max']})) #проверим корректность разделения

                total_income         
                         min      max
income_group_id                      
1                     195543  2265604
2                     156400   195537
3                     107798   156373
4                      20667   107789


Создадим столбец с обозначениями групп по доходу.

In [34]:
def income_group (income_group_id):
    if income_group_id == 1:
        return 'Высокий'
    if income_group_id == 2:
        return 'Выше среднего'
    if income_group_id == 3:
        return 'Ниже среднего'
    if income_group_id == 4:
        return 'Низкий'
data['income_group'] = data['income_group_id'].apply(income_group)
print(data['income_group'].value_counts())

Высокий          5382
Выше среднего    5381
Ниже среднего    5381
Низкий           5381
Name: income_group, dtype: int64


Создадим словарь с группами по доходу.

In [35]:
income_group_dict = data[['income_group','income_group_id']]
income_group_dict = income_group_dict.drop_duplicates().reset_index(drop=True)
print(income_group_dict.sort_values('income_group_id'))

    income_group  income_group_id
0        Высокий                1
2  Выше среднего                2
1  Ниже среднего                3
3         Низкий                4


## Задание 2. Анализ данных

### Анализ зависимости между различными параметрами клиентов и возвратом кредита в срок

Для начала создадим датафрейм с теми столбцами, которые нам понадобятся для дальнейшего анализа.  
Дополнительно будем использовать словари

In [36]:
debt_return = data[['children_group', 'family_status_id', 'income_group_id', 'purpose_id', 'debt']]
print(debt_return.head(10))

  children_group  family_status_id  income_group_id  purpose_id  debt
0      Есть дети                 0                1           1     0
1      Есть дети                 0                3           3     0
2      Нет детей                 0                3           1     0
3      Есть дети                 0                1           4     0
4      Нет детей                 1                2           2     0
5      Нет детей                 1                1           1     0
6      Нет детей                 0                1           1     0
7      Нет детей                 0                3           4     0
8      Есть дети                 1                4           2     0
9      Нет детей                 0                3           1     0


Посмотрим на долю невозвращенных долгов по каждой группе.

Проанализируем данные по долгам в разрезе **уровня дохода**.

In [37]:
debt_return = debt_return.merge(income_group_dict, on='income_group_id', how='left')#подтягиваем словарь с обозначением групп по доходу
print(debt_return.head(10))

  children_group  family_status_id  income_group_id  purpose_id  debt  \
0      Есть дети                 0                1           1     0   
1      Есть дети                 0                3           3     0   
2      Нет детей                 0                3           1     0   
3      Есть дети                 0                1           4     0   
4      Нет детей                 1                2           2     0   
5      Нет детей                 1                1           1     0   
6      Нет детей                 0                1           1     0   
7      Нет детей                 0                3           4     0   
8      Есть дети                 1                4           2     0   
9      Нет детей                 0                3           1     0   

    income_group  
0        Высокий  
1  Ниже среднего  
2  Ниже среднего  
3        Высокий  
4  Выше среднего  
5        Высокий  
6        Высокий  
7  Ниже среднего  
8         Низкий  
9  Ниж

In [38]:
income_group_debt = debt_return.groupby('income_group').agg({'debt':['count', 'sum']}) #создаем датафрейм, сгруппированный по уровню дохода, с новыми столбцами с информацией об общем количестве клиентов в каждой группе и количестве клиентов с просроченным возвратом долга
income_group_debt['ratio'] = income_group_debt['debt']['sum'] / income_group_debt['debt']['count'] #создаем столбец с долей должников
print(income_group_debt)

               debt          ratio
              count  sum          
income_group                      
Высокий        5382  386  0.071721
Выше среднего  5381  456  0.084743
Ниже среднего  5381  472  0.087716
Низкий         5381  427  0.079353


Результаты достаточно интересны: наиболее ответственны в плане возврата кредитов клиенты с высоким доходом (что понятно, так как им достаточно просто расплачиваться по кредитным обязательствам) и низким.  
Второй результат достаточно неожиданный: вероятно, клиенты с низким доходом стремятся не попасть в "долговую яму" и, взяв кредит, тщательно следят за выплатами.

Проанализируем данные по долгам в разрезе **наличия детей**.

In [39]:
children_group_debt = debt_return.groupby('children_group').agg({'debt':['count', 'sum']})
children_group_debt['ratio'] = children_group_debt['debt']['sum'] / children_group_debt['debt']['count'] 
print(children_group_debt)

                 debt           ratio
                count   sum          
children_group                       
Есть дети        7376   678  0.091920
Нет детей       14149  1063  0.075129


Итак, заемщики с детьми менее надежны, чем бездетные. У них одна надежда: что у кредитного менеджера тоже есть дети...

Проанализируем данные по долгам в разрезе **семейного положения**.

In [40]:
debt_return = debt_return.merge(family_status_dict, on='family_status_id', how='left')
family_status_debt = debt_return.groupby('family_status').agg({'debt':['count', 'sum']})
family_status_debt['ratio'] = family_status_debt['debt']['sum'] / family_status_debt['debt']['count'] 
print(family_status_debt)

                        debt          ratio
                       count  sum          
family_status                              
Не женат / не замужем   2813  274  0.097405
в разводе               1195   85  0.071130
вдовец / вдова           960   63  0.065625
гражданский брак        4177  388  0.092890
женат / замужем        12380  931  0.075202


Как и ожидалось, четкие критерии для категоризации клиентов по семейному положению выработать сложно.  
Наименее надежные категории - холостые/незамужние и состоящие в гражданском браке.  
Лучше всего отдают кредиты - вдовы/вдовцы, чуть похуже - разведенные и состоящие в официальном браке

Проанализируем данные по долгам в разрезе **целей кредита**.

In [41]:
debt_return = debt_return.merge(purpose_dict, on='purpose_id', how='left')
purpose_group_debt = debt_return.groupby('purpose_group').agg({'debt':['count', 'sum']})
purpose_group_debt['ratio'] = purpose_group_debt['debt']['sum'] / purpose_group_debt['debt']['count'] 
print(purpose_group_debt)

                                       debt          ratio
                                      count  sum          
purpose_group                                             
образование                            4022  370  0.091994
приобретение автомобиля                4315  403  0.093395
приобретение или ремонт недвижимости  10840  782  0.072140
свадьба                                2348  186  0.079216


Хуже всего отдают кредит те, у кто берут его для получения образования и приобретения автомобиля.  
Заемщики, приобретающие недвижимость или нуждающиеся в кредите для проведения свадьбы, более надежны.

### Результаты анализа

После анализа данных можно сделать следующие выводы.  
1. Кредитоспособность заемщиков связана с целым рядом различных параметров.
2. Во-первых, выяснилось, что более надежны в плане своевременного возврата кредита заемщики без детей. Очевидно, что это связано с более высокой финансовой нагрузкой, которую испытывают семьи с детьми.
3. Во-вторых, нет однозначной связи между уровнем дохода и возвратом кредита в срок.    
  A. Наиболее ответственны в плане возврата кредитов клиенты с высоким доходом, что объяснимо, так как им проще расплачиваться по кредитным обязательствам.  
  B. Интересно, что на втором месте по надежности оказались заещики с низким доходом. Вероятно, такие клиенты стремятся не попасть в "долговую яму" и, взяв кредит, тщательно следят за выплатами.
4. Как и ожидалось, четкие критерии для категоризации клиентов по семейному положения выработать сложно.  
  A. Наименее надежные категории - холостые/незамужние и состоящие в гражданском браке. По всей видимости, это наиболее молодые и наименее ответственные сегменты заемщиков.  
  B. Лучше всего отдают кредиты - вдовы/вдовцы, что, по всей видимости, связано с высоким уровнем финансовой ответственности старшего поколения.  
  C. Чуть менее надежны, чем вдовы, - разведенные и состоящие в официальном браке. Сам факт регистрации отношений (пусть и в прошлом), возможно, связан с уровнем ответственности клиента.  
5. С точки зрения целей кредита, можно сформулировать следующее утверждение: более надежны те заемщики, которые берут кредит, на цели, связанные с созданием семьи.  
  A. Меньше всего просроченных кредитов среди тех, кто взял заем на покупку недвижимости или организацию свадьбы. Понятно, что среди покупателей недвижимости есть не только семейные, тем не менее, определенная взаимосвязь существует.  
  B. Хуже всего отдают кредит те, у кто берут его для получения образования и приобретения автомобиля.